In [1]:
import random
import numpy as np

%run ANN.ipynb
%run kNN.ipynb
%run utils.ipynb

In [1]:
class AugmentImages:
    def __init__(self, is_neural):
        self.is_neural = is_neural

    # This shuffles the data with rotated images while keep values to targets intact
    def shuffle_data(self, x, y, amount):
        data = list(zip(x, y))
        random.shuffle(data)
        x, y = zip(*data[:amount])
        return (x, y)

    def augment_data(self, values, labels, rotation):
        augmented_images = []
        augmented_labels = []

        # loop over values to rotate images clockwise
        for num in range(0, len(values)):
            #angle = random.randint(0, rotation) # random angle within specified range
            angle = rotation
            acw_img = rotate_image(values[num], angle)[0]
            augmented_images.append(acw_img)
            augmented_labels.append(labels[num])

        # loop over values to rotate images anticlockwise
        for num in range(0, len(values)):
            #angle = random.randint(0, rotation) # random angle within specified range
            angle = rotation
            cw_img = rotate_image(values[num], angle)[1]
            augmented_images.append(cw_img)
            augmented_labels.append(labels[num])

        ## check what to return depending on if using ANN or k-NN
        if self.is_neural:
            return {
                "images": np.asfarray(augmented_images),
                "labels": np.asfarray(augmented_labels)
            }
        else:
            return {
                "images": np.asfarray(augmented_images),
                "labels": augmented_labels
            }
    
    # Custom method to train either ANN or k-NN with rotated images
    def fit_augmented_model(self, trainX, trainY, testX, testY, shuffle_amount, degree):
        data = self.augment_data(trainX, trainY, degree)

        images = data['images']
        labels = data['labels']

        for num in range(0, len(images)):
            trainX.append(images[num])

        for num in range(0, len(labels)):
            trainY.append(labels[num])

        print("Shuffling data...")
        trainX, trainY = self.shuffle_data(trainX, trainY, shuffle_amount)

        if self.is_neural:
            n = NeuralNetwork(epochs=50, learningrate=0.3, batch_size=1) # using optimal hyperparameters
            n.train(trainX, trainY)
            n.test(testX, testY)
            accuracy = n.evaluate(n.results)
        else:
            knn = kNN(trainX, trainY, k=3, weighted=False, sim=cosine_similarity) # using optimal hyperparameters
            knn.test(testX, testY)
            accuracy = knn.evaluate(knn.results)

        return accuracy
